In [51]:
import json
import numpy as np
import pandas as pd
from operator import itemgetter
from scipy.spatial.distance import pdist, squareform
from random import *
import math
#avg_ratings = pd.read_csv('/Users/Joyce/Desktop/Personalization Theory/project/Yelp/average_ratings.csv',index_col=0)
original_data=pd.read_csv('/Users/Joyce/Desktop/Personalization Theory/project/Yelp/rating data 135.csv',index_col=0)
#original_data=pd.read_csv('/Users/Joyce/Desktop/Personalization Theory/project/Yelp/rating data 125.csv',index_col=0)
#original_data=pd.read_csv('/Users/Joyce/Desktop/Personalization Theory/project/Yelp/rating data 115.csv',index_col=0)
#original_data=pd.read_csv('/Users/Joyce/Desktop/Personalization Theory/project/Yelp/rating data 105.csv',index_col=0)
#original_data=pd.read_csv('/Users/Joyce/Desktop/Personalization Theory/project/Yelp/rating data 95.csv',index_col=0)

split non-empty pairs data into 80% train and 20% test sets

In [41]:
import random
test=[]
tune=[]
train=[]
random.seed(10)
for j in original_data.columns:
    for i in original_data.index:
        r=random.random()
        if not pd.isnull(original_data[j][i]):
            if r<=0.6:
                train.append([j,i,original_data[j][i]])
            elif r>0.6 and r<=0.8:
                tune.append([j,i,original_data[j][i]])
            else:
                test.append([j,i,original_data[j][i]])
data_train=original_data.copy()
for item in tune:
    if item[0] in data_train.columns and item[1] in data_train.index:
        data_train[item[0]][item[1]]=np.nan
for item in test:
    if item[0] in data_train.columns and item[1] in data_train.index:
        data_train[item[0]][item[1]]=np.nan

calculate similarity matrix of restaurants in the training dataset

In [42]:
#Step 1: calculate the mean centered rating matrix from the original dataset
mean_ratings=data_train.mean(axis=0,skipna=True)
substracted_data=data_train.copy()
for j in substracted_data.columns:
    for i in substracted_data.index:
        if not pd.isnull(substracted_data[j][i]):
            substracted_data[j][i]=substracted_data[j][i]-mean_ratings[j]
#Step 2: function to compute similarity of any pair of 2 restaurantes
def adjusted_cosine(u, v):
    m = u.shape[0]
    udotv = 0
    u_norm = 0
    v_norm = 0
    for i in range(m):
        if (np.isnan(u[i])) or (np.isnan(v[i])):
            continue
            
        udotv += u[i] * v[i]
        u_norm += u[i] * u[i]
        v_norm += v[i] * v[i]
        
    u_norm = np.sqrt(u_norm)
    v_norm = np.sqrt(v_norm)
    
    if (u_norm == 0) or (v_norm == 0):
        ratio = 1.0
    else:
        ratio = udotv / (u_norm * v_norm)
    return ratio
#Step 3: create the similarity matrix
items=substracted_data.columns
similarity=pd.DataFrame(index=items, columns=items)
for i in range(len(items)):
    for j in range(len(items)):
        similarity.ix[i][j] = adjusted_cosine(substracted_data.ix[:,i],substracted_data.ix[:,j])

item-based model with neighborhood size=t

In [43]:
def item_based(dataset,similarity,t):
    data=dataset.copy()
    for u in range(len(data.index)):
        #get list of rated restaurent and its rating and unrated restaurants
        rated_restaurants={}
        unrated_id=[]
        for i in range(len(data.ix[u])):
            id_r=data.iloc[u].index[i]
            if not pd.isnull(data.ix[u][i]):
                if id_r not in rated_restaurants:
                    rated_restaurants[id_r]=data.ix[u][i]
            if pd.isnull(data.ix[u][i]):
                    unrated_id.append(id_r)

        #fill predicted ratings for unrated restaurents based on similarity and rating of rated restaurants
        unrated_restaurants={}
        unrated_restaurants=unrated_restaurants.fromkeys(unrated_id, None)
        for k in unrated_restaurants:
            neighbors={}
            for j in rated_restaurants:
                if similarity[k][j]>=0:
                    if j not in neighbors:
                        neighbors[j]=similarity[k][j]
            neighbors=sorted(neighbors.items(),key=itemgetter(1),reverse=True)
            nominator=0
            denominator=0
            #if no neighbour,just use the average rating of the restaurant by other users from the dataset
            if len(neighbors)==0: 
                unrated_restaurants[k]=mean_ratings[k]
            elif 0<len(neighbors)& len(neighbors)<t:
                for n in neighbors:
                    nominator+=n[1]*rated_restaurants[n[0]]
                    denominator+=abs(n[1])
                    #add a very small random number for recommendation selection in case many restaurants will have the same rating
                    unrated_restaurants[k]=nominator/denominator+random()/100000
            else:
                for n in neighbors[:t]:
                    nominator+=n[1]*rated_restaurants[n[0]]
                    denominator+=abs(n[1])
                    unrated_restaurants[k]=nominator/denominator

        #fill in unrated restaurents ratings
        for i in range(len(data.ix[u])):
            if pd.isnull(data.ix[u][i]):
                data.ix[u][i]=unrated_restaurants[''.join(data.ix[u,i:i+1].index.tolist())]
    return data

for neighborhood size from 1 to 20, calculate and compare the RMSE for each model to select the final model

In [101]:
from random import *
rmse_list=[]
for t in range(1,21):
    prediction_train=item_based(dataset=data_train,similarity=similarity,t=t)
    SUM=0
    COUNT=0
    for item in tune:
        if item[0] in prediction_train.columns and item[1] in prediction_train.index:
            SUM+=(prediction_train[item[0]][item[1]]-item[2])**2
            COUNT+=1
    RMSE=math.sqrt(SUM/COUNT)
    rmse_list.append([t,RMSE])

In [55]:
rmse_list

NameError: name 'rmse_list' is not defined

In [ ]:
rmse_list=[
 [1, 1.3071464370353465],
 [2, 1.2126399065206135],
 [3, 1.184821847350271],
 [4, 1.17615928536058],
 [5, 1.1703095535923995],
 [6, 1.1671322967069724],
 [7, 1.1661703221368462],
 [8, 1.1660491128683121],
 [9, 1.1657912761767009],
 [10, 1.1657348748057679],
 [11, 1.1657992576527165],
 [12, 1.165769982276961],
 [13, 1.1657770431928116],
 [14, 1.1656986794703161],
 [15, 1.1657180030316867],
 [16, 1.1657127691870086],
 [17, 1.1656567683132888],
 [18, 1.1656319921815093],
 [19, 1.1656324072669737],
 [20, 1.1656275549944415]]

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
plot_data = np.array(rmse_list)
x, y = plot_data.T
plt.plot(x,y)
plt.scatter(x,y)
plt.xlabel('Neighorhood Size')
plt.ylabel('RMSE')
plt.show()
plt.close

RMSE varies little when we increase neighborhood size from 10 to 20.
Therefore, we choose neighborhood size t=10 to be the best neighborhood size to avoid overfitting
calculate rmse on the test dataset

In [44]:
#calculate rmse on the test dataset
from random import *
prediction_train=item_based(dataset=data_train,similarity=similarity,t=10)
SUM=0
COUNT=0
for item in test:
    if item[0] in prediction_train.columns and item[1] in prediction_train.index:
        if not pd.isnull(prediction_train[item[0]][item[1]]):
            SUM+=(prediction_train[item[0]][item[1]]-item[2])**2
            COUNT+=1
test_RMSE=math.sqrt(SUM/COUNT)

Coverage of the model:
Calculate and compare RMSE when increasing the size of the dataset

In [45]:
#test_RMSE135=test_RMSE
#test_RMSE125=test_RMSE
#test_RMSE115=test_RMSE
#test_RMSE105=test_RMSE
#test_RMSE95=test_RMSE

In [46]:
#test_RMSE135,test_RMSE125,test_RMSE115,test_RMSE105,test_RMSE95

(1.2069856768804772,
 1.1858021424127776,
 1.1919861843762687,
 1.1650238390666052,
 1.1686472176226619)

In [54]:
coverage_RMSEs=[1.2069856768804772, 1.1858021424127776, 1.1919861843762687,1.1650238390666052,1.1686472176226619]
#test_RMSE135,test_RMSE125,test_RMSE115,test_RMSE105,test_RMSE95 dataset size
coverage_size=[387300, 498491, 595620, 722455, 848598]
from matplotlib import pyplot as plt
plt.plot(coverage_size,coverage_RMSEs)
plt.scatter(coverage_size,coverage_RMSEs)
plt.xlabel('Dataset Size')
plt.ylabel('RMSE')
plt.show()
plt.close

<function matplotlib.pyplot.close>